In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://www.pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html.read(), 'html.parser')
namelist = bs.find_all('span', {'class': 'green'})

for name in namelist:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


In [12]:
namelist = bs.find_all(string='the prince')
print(len(namelist))

7


In [14]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
for child in bs.find('table', {'id': 'giftList'}).children:
    print(child)



<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>


<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>


In [15]:
for sibling in bs.find('table', {'id': 'giftList'}).tr.next_siblings:
    print(sibling)



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

In [16]:
print(bs.find('img', {'src': '../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())


$15.00



In [17]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
images = bs.find_all('img', {'src': re.compile('..\/img\/gifts/img.*.jpg')})

for image in images:
    print(image['src'])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


In [26]:
print(bs.find_all(lambda tag: tag.get_text() == 'Or maybe he\'s only resting?'))
print(bs.find_all('span', string='Or maybe he\'s only resting?'))

[<span class="excitingNote">Or maybe he's only resting?</span>]
[<span class="excitingNote">Or maybe he's only resting?</span>]


In [18]:
import random
from urllib.error import HTTPError
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup


def get_internal_links(bs, url):
    '''Retrieves a list of all Internal links found on a page'''
    netloc = urlparse(url).netloc
    scheme = urlparse(url).scheme
    internal_links = set()
    for link in bs.find_all('a'):
        if not link.attrs.get('href'):
            continue
        parsed = urlparse(link.attrs['href'])
        if parsed.netloc == '':
            internal_links.add(f'{scheme}://{netloc}/{link.attrs["href"].strip("/")}')
        elif parsed.netloc == netloc:
            internal_links.add(link.attrs['href'])
    return list(internal_links)
            

def get_external_links(bs, url):
    '''Retrieves a list of all external links found on a page'''
    netloc = urlparse(url).netloc
    external_links = set()
    for link in bs.find_all('a'):
        if not link.attrs.get('href'):
            continue
        parsed = urlparse(link.attrs['href'])
        if parsed.netloc != '' and parsed.netloc != netloc:
            external_links.add(link.attrs['href'])
    return list(external_links)

def get_random_external_link(starting_page):
    bs = BeautifulSoup(urlopen(starting_page), 'html.parser')
    external_links = get_external_links(bs, starting_page)
    if not len(external_links):
        print('No external links, looking around the site for one')
        internal_links = get_internal_links(bs, starting_page)
        return get_random_external_link(random.choice(internal_links))
    else:
        return random.choice(external_links)
    
def follow_external_only(starting_site):
    try:
        external_link = get_random_external_link(starting_site)
        print(f'Random external link is: {external_link}')
        follow_external_only(external_link)
    except ValueError as e:
        print('Error value link: ', e)
    except HTTPError as e:
        print('Error http: ', e)
    except TimeoutError as e:
        print('Connection timeout: ', e)
    except IndexError as e:
        print('Range index not found: ', e)


follow_external_only('https://pythonworld.ru/')

Random external link is: https://pyplanet.ru/article/telegram-bot-aiogram-quickstart.html
Random external link is: https://zallj.com/g/qqb62fps0r99b979a60661cec322b98fba714163/?erid=5jtCeReNwxHpfQTDve31wmc
Random external link is: https://yandex.cloud/ru/services/smartcaptcha?utm_source=captcha&utm_medium=chbx&utm_campaign=security
Random external link is: https://yandex.ru/legal/cloud_termsofuse
Random external link is: https://yandex.cloud
Random external link is: https://sourcecraft.dev/
Random external link is: https://ya.ru
Random external link is: https://yandex.ru/support/smart-captcha/problems.html?form-unique_key=7639461546662057535&form-fb-hint=2.47
Error http:  HTTP Error 404: Not Found


In [ ]:
# Collects a list of all external URLs found on the site
all_ext_links = []
all_int_links = []


def get_all_external_links(url):
    bs = BeautifulSoup(urlopen(url), 'html.parser')
    internal_links = get_internal_links(bs, url)
    external_links = get_external_links(bs, url)
    for link in external_links:
        if link not in all_ext_links:
            all_ext_links.append(link)
            print(link)

    for link in internal_links:
        if link not in all_int_links:
            all_int_links.append(link)
            get_all_external_links(link)


all_int_links.append('https://oreilly.com')
get_all_external_links('https://www.oreilly.com/')

In [27]:
import requests

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        

def get_page(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser')

def scrape(url):
    bs = get_page(url)
    title = bs.find('h1').text
    lines = bs.select('div.StoryBodyCompanionColumn div p')
    body = '\n'.join(line.text for line in lines)
    return Content(url, title, body)


url = 'https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/'

content = scrape(url)
print(f'Title: {content.title}')
print(f'URL: {content.url}\n')
print(content.body)

Title: Delivering inclusive urban access: 3 uncomfortable truths
URL: https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/




In [64]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
    
    def print_info(self):
        print('TITLE:', f'{self.title}'.replace("\n", "").replace(" ", ""))
        print(f'URL: {self.url}')
        print('BODY:', f'{self.body}'.replace("\n", "").strip("\t"))

def scrapeCNN(url):
    bs = BeautifulSoup(urlopen(url))
    title = bs.find('h1').text
    body = bs.find('div', {'class': 'headline__wrapper'}).text
    print('body: ')
    print(body)
    return Content(url, title, body)

def scrapeBrookings(url):
    bs = BeautifulSoup(urlopen(url))
    title = bs.find('h1').text
    body = bs.find('div', {'class': 'label-large'}).text
    return Content(url, title, body)

url = 'https://www.brookings.edu/research/robotic-rulemaking/'
content = scrapeBrookings(url)
content.print_info()
print()
url = 'https://www.cnn.com/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html'
content = scrapeCNN(url)
content.print_info()

TITLE: Roboticrulemaking
URL: https://www.brookings.edu/research/robotic-rulemaking/
BODY: More On

body: 



      Dogecoin jumps after Elon Musk replaces Twitter bird with Shiba Inu
    

TITLE: DogecoinjumpsafterElonMuskreplacesTwitterbirdwithShibaInu
URL: https://www.cnn.com/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html
BODY:       Dogecoin jumps after Elon Musk replaces Twitter bird with Shiba Inu    


In [76]:
import requests
from bs4 import BeautifulSoup as BS

class Content:
    """Общий родительский класс для всех статей/страниц."""
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        
    def print_info(self):
        """Гибкая функция печати, управляющая выводом данных."""
        print(f'URL: {self.url}')
        print(f'TITLE: {self.title}')
        print(f'BODY:\n{self.body}')
        

class Website:
    """Содержит информацию о структуре сайта."""
    def __init__(self, name, url, title_tag, body_tag):
        self.name = name
        self.url = url
        self.title_tag = title_tag
        self.body_tag = body_tag
        

class Crawler:
    def get_page(self, url):
        try:
            response = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BS(response.text, 'html.parser')
    
    def safe_get(self, bs, selector):
        """
        Служебная функция, используемая для получения строки 
        содержимого из объекта BeautifulSoup и селектора.
        Если объект для данного селектора не найден,
        то возвращает пустую строку.
        """
        selected_elems = bs.select(selector)
        if selected_elems is not None and len(selected_elems) > 0:
            return '\n'.join(elem.get_text() for elem in selected_elems)
        return ''
    
    def get_content(self, website, path):
        """Извлекает содержимое страницы с заданным URL."""
        url = website.url + path
        bs = self.get_page(url)
        if bs is not None:
            title = self.safe_get(bs, website.title_tag)
            body = self.safe_get(bs, website.body_tag)
            return Content(url, title, body)
        return Content(url, '', '')


crawler = Crawler()

site_data = [
    ['O\'Reilly Media', 'https://www.oreilly.com', 'h1', 'div.title-description'],
    ['Reuters', 'https://www.reuters.com', 'h1', 'div.ArticleBodyWrapper'],
    ['Brookings', 'https://www.brookings.edu', 'h1', 'div.post-body'],
    ['CNN', 'https://www.cnn.com', 'h1', 'div.article__content']
]

websites = []

for name, url, title, body in site_data:
    websites.append(Website(name, url, title, body))
    
crawler.get_content(websites[0], '/library/view/web-scraping-with/9781491910283').print_info()
crawler.get_content(websites[1], '/article/us-usa-epa-pruitt-idUSKBN19W2D0').print_info()
crawler.get_content(websites[2], '/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/').print_info()
crawler.get_content(websites[3], '/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html').print_info()

URL: https://www.oreilly.com/library/view/web-scraping-with/9781491910283
TITLE: Web Scraping with Python
BODY:

URL: https://www.reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0
TITLE: 
BODY:

URL: https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/
TITLE: Idea to Retire: Old methods of policy education
BODY:

URL: https://www.cnn.com/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html
TITLE: 
      Dogecoin jumps after Elon Musk replaces Twitter bird with Shiba Inu
    
BODY:



New York
CNN
         — 
    


            Twitter’s traditional bird icon was booted and replaced with an image of a Shiba Inu, an apparent nod to dogecoin, the joke cryptocurrency that CEO Elon Musk is being sued over. 
    

            Musk addressed the change Monday afternoon, tweeting, “as promised” above an image of a year-old conversation in which another user suggested that Musk “just buy Twitter” and “change the bird logo to a doge.” 
    

